In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
from tensorflow.keras.applications import VGG16
conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))

In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import shutil

In [ ]:
original_data = "/content/drive/My Drive/Datasets/Glasses and tables/Original Data"
working_data = "/content/drive/My Drive/Datasets/Glasses and tables/Working Data"

train_dir = os.path.join(working_data, 'train')
os.mkdir(train_dir)
validation_dir = os.path.join(working_data, 'validation')
os.mkdir(validation_dir)
test_dir = os.path.join(working_data, 'test')
os.mkdir(test_dir)


In [ ]:
train_glass_dir = os.path.join(train_dir, 'glasses')
os.mkdir(train_glass_dir)
train_table_dir = os.path.join(train_dir, 'tables')
os.mkdir(train_table_dir)

validation_glass_dir = os.path.join(validation_dir, 'glasses')
os.mkdir(validation_glass_dir)
validation_tables_dir = os.path.join(validation_dir, 'tables')
os.mkdir(validation_tables_dir)



In [ ]:
list_of_data = os.listdir(original_data)
glasses = []
for i in list_of_data:
  if "glass" in i:
    glasses.append(i)
  else:
    None

In [ ]:
len(glasses)

3203

Training And Validation Glasses

In [ ]:
glasses_train = glasses [0:1000]
glasses_valid = glasses [1000:1500]
glasses_test = glasses [1500:2000]

In [ ]:
print(len(glasses_train))
print(len(glasses_valid))
print(len(glasses_test))

1000
500
500


In [ ]:
for k in glasses_train:
  src = os.path.join(original_data, k)
  dst = os.path.join(train_glass_dir, k)
  shutil.copyfile(src, dst) 
  

In [ ]:
path = "/content/drive/My Drive/Datasets/Glasses and tables/Working Data/train/glasses"
files = os.listdir(path)
len(files)

1000

In [ ]:
for l in glasses_valid:
  src = os.path.join(original_data, l)
  dst = os.path.join(validation_glass_dir, l)
  shutil.copyfile(src, dst)

In [ ]:
path = "/content/drive/My Drive/Datasets/Glasses and tables/Working Data/validation/glasses"
files = os.listdir(path)
len(files)


500

Training And Validation Tables

In [ ]:
list_of_data = os.listdir(original_data)
tables = []
for i in list_of_data:
  if "table" in i:
    tables.append(i)
  else:
    None

In [ ]:
len(tables)

3204

In [ ]:
tables_train = tables [0:1000]
tables_valid = tables [1000:1500]

In [ ]:
len(tables_train)

1000

In [ ]:
len(tables_valid)

500

In [ ]:
for k in tables_train:
  src = os.path.join(original_data, k)
  dst = os.path.join(train_table_dir, k)
  shutil.copyfile(src, dst) 

In [ ]:
path = "/content/drive/My Drive/Datasets/Glasses and tables/Working Data/train/tables"
files = os.listdir(path)
len(files)

1000

In [ ]:
for l in tables_valid:
  src = os.path.join(original_data, l)
  dst = os.path.join(validation_tables_dir, l)
  shutil.copyfile(src, dst)

In [ ]:
path = "/content/drive/My Drive/Datasets/Glasses and tables/Working Data/validation/tables"
files = os.listdir(path)
len(files)

500

In [ ]:
train_dir = "/content/drive/My Drive/Datasets/Glasses and tables/Working Data/train"
validation_dir = "/content/drive/My Drive/Datasets/Glasses and tables/Working Data/validation"

In [ ]:
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

In [ ]:
def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 4, 4, 512))
    labels = np.zeros(shape=(sample_count))
    generator = datagen.flow_from_directory(
        directory,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')
    
    i=0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        
      
        if i * batch_size >= sample_count:
            break
        return features, labels

In [ ]:
train_features, train_labels = extract_features(train_dir, 2000)
validation_features, validation_labels = extract_features(validation_dir, 1000)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [ ]:
train_features = np.reshape(train_features, (2000, 4*4* 512))
validation_features = np.reshape(validation_features, (1000, 4*4* 512))

In [ ]:
from keras import models
from keras import layers
from keras import optimizers
model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_dim=4 * 4 * 512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer=optimizers.RMSprop(lr=2e-5),
loss='binary_crossentropy',
metrics=['acc'])
history = model.fit(train_features, train_labels,
epochs=30,
batch_size=20,
validation_data=(validation_features, validation_labels))

Using TensorFlow backend.


Train on 2000 samples, validate on 1000 samples
Epoch 1/30
2000/2000 [==============================] - 1s 422us/step - loss: 0.6900 - acc: 0.9950 - val_loss: 0.6880 - val_acc: 0.9910
Epoch 2/30
2000/2000 [==============================] - 1s 312us/step - loss: 0.6838 - acc: 0.9975 - val_loss: 0.6798 - val_acc: 0.9960
Epoch 3/30
2000/2000 [==============================] - 1s 294us/step - loss: 0.6743 - acc: 0.9995 - val_loss: 0.6713 - val_acc: 0.9960
Epoch 4/30
2000/2000 [==============================] - 1s 290us/step - loss: 0.6677 - acc: 0.9990 - val_loss: 0.6626 - val_acc: 0.9960
Epoch 5/30
2000/2000 [==============================] - 1s 288us/step - loss: 0.6583 - acc: 0.9995 - val_loss: 0.6538 - val_acc: 0.9980
Epoch 6/30
2000/2000 [==============================] - 1s 322us/step - loss: 0.6508 - acc: 0.9990 - val_loss: 0.6450 - val_acc: 0.9980
Epoch 7/30
2000/2000 [==============================] - 1s 311us/step - loss: 0.6420 - acc: 0.9990 - val_loss: 0.6361 - val_acc: 0.9980


In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import regularizers
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))


In [ ]:
conv_base.trainable = False

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers


datagen = ImageDataGenerator(
rescale=1./255,
rotation_range=40,
width_shift_range=0.2,
height_shift_range=0.2,
shear_range=0.2,
zoom_range=0.2,
horizontal_flip=True,
fill_mode='nearest')

In [ ]:
train_generator = datagen.flow_from_directory(
                    train_dir,
                    target_size=(150, 150),
                    batch_size=20,
                    class_mode='binary'
)

Found 2000 images belonging to 2 classes.


In [ ]:

validation_generator = datagen.flow_from_directory(
                        validation_dir,
                        target_size=(150, 150),
                        batch_size=20,
                        class_mode='binary'
)

Found 1000 images belonging to 2 classes.


In [ ]:
model.compile(loss='binary_crossentropy',
optimizer=optimizers.RMSprop(lr=2e-5),
metrics=['acc'])

In [ ]:
history = model.fit_generator(
                                train_generator,
                                steps_per_epoch=50,
                                epochs=20,
                                validation_data=validation_generator,
                                validation_steps=50
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/20
50/50 [==============================] - 364s 7s/step - loss: 0.5577 - acc: 0.7210 - val_loss: 0.3901 - val_acc: 0.9120
Epoch 2/20
50/50 [==============================] - 107s 2s/step - loss: 0.3870 - acc: 0.8490 - val_loss: 0.2978 - val_acc: 0.9170
Epoch 3/20
34/50 [===================>..........] - ETA: 18s - loss: 0.3257 - acc: 0.8809

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


50/50 [==============================] - 77s 2s/step - loss: 0.3178 - acc: 0.8900 - val_loss: 0.2433 - val_acc: 0.9260
Epoch 4/20
50/50 [==============================] - 61s 1s/step - loss: 0.2384 - acc: 0.9210 - val_loss: 0.2146 - val_acc: 0.9230
Epoch 5/20
50/50 [==============================] - 47s 940ms/step - loss: 0.2269 - acc: 0.9070 - val_loss: 0.1830 - val_acc: 0.9460
Epoch 6/20
50/50 [==============================] - 44s 879ms/step - loss: 0.2145 - acc: 0.9140 - val_loss: 0.1703 - val_acc: 0.9380
Epoch 7/20
50/50 [==============================] - 41s 829ms/step - loss: 0.1851 - acc: 0.9390 - val_loss: 0.1619 - val_acc: 0.9430
Epoch 8/20
50/50 [==============================] - 40s 797ms/step - loss: 0.1659 - acc: 0.9390 - val_loss: 0.1571 - val_acc: 0.9480
Epoch 9/20
50/50 [==============================] - 39s 786ms/step - loss: 0.1723 - acc: 0.9450 - val_loss: 0.1458 - val_acc: 0.9470
Epoch 10/20
50/50 [==============================] - 39s 781ms/step - loss: 0.1501 - a

In [ ]:
model.save("/content/drive/My Drive/Saved_Models/Glass&Tables.h5")

In [ ]:
conv_base.trainable = True
set_trainable = False
for layer in conv_base.layers:
  if layer.name == 'block5_conv1':
    set_trainable = True
  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False

In [ ]:
import tensorflow as tf

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_acc')>0.99 and logs.get('val_loss')<0.04):
            print("\nReached 99% accuracy so cancelling training!")
            self.model.stop_training = True

In [ ]:
callbacks = myCallback()

In [ ]:
model.compile(loss='binary_crossentropy',
optimizer=optimizers.RMSprop(lr=1e-5),
metrics=['acc'])


history = model.fit_generator(
train_generator,
steps_per_epoch=80,
epochs=50,
callbacks=[callbacks],
validation_data=validation_generator,
validation_steps=40)

Epoch 1/50
80/80 [==============================] - 48s 602ms/step - loss: 0.0165 - acc: 0.9956 - val_loss: 0.0466 - val_acc: 0.9837
Epoch 2/50
73/80 [==========================>...] - ETA: 2s - loss: 0.0160 - acc: 0.9932

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


80/80 [==============================] - 48s 605ms/step - loss: 0.0149 - acc: 0.9937 - val_loss: 0.0658 - val_acc: 0.9800
Epoch 3/50
80/80 [==============================] - 49s 608ms/step - loss: 0.0233 - acc: 0.9912 - val_loss: 0.0325 - val_acc: 0.9887
Epoch 4/50
80/80 [==============================] - 49s 614ms/step - loss: 0.0151 - acc: 0.9944 - val_loss: 0.0662 - val_acc: 0.9825
Epoch 5/50
80/80 [==============================] - 48s 604ms/step - loss: 0.0201 - acc: 0.9950 - val_loss: 0.0469 - val_acc: 0.9812
Epoch 6/50
80/80 [==============================] - 48s 599ms/step - loss: 0.0115 - acc: 0.9962 - val_loss: 0.0427 - val_acc: 0.9812
Epoch 7/50
80/80 [==============================] - 48s 604ms/step - loss: 0.0210 - acc: 0.9894 - val_loss: 0.0830 - val_acc: 0.9775
Epoch 8/50
80/80 [==============================] - 49s 609ms/step - loss: 0.0178 - acc: 0.9944 - val_loss: 0.0459 - val_acc: 0.9862
Epoch 9/50
80/80 [==============================] - 48s 604ms/step - loss: 0.019

In [ ]:
import tensorflow.keras
from keras import models


In [ ]:
model = models.load_model("Glass&TablesFinal.h5")

Testing The Model

In [ ]:
img_path = "/content/drive/My Drive/Datasets/Glasses and tables/Working Data/test/table4.jpg"
from keras.preprocessing import image
import numpy as np
img = image.load_img(img_path, target_size=(150, 150))
img_tensor = image.img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis=0)
img_tensor /= 255.


In [ ]:
a = model.predict(img_tensor)

In [ ]:
if a < 0.5:
  print("Glass")

else:
  print("Table")


Table
